<a href="https://colab.research.google.com/github/rafalsza/moje/blob/master/wavetrend_backtest.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install vectorbt
!pip install python-binance
!pip install pandas-ta

In [3]:
import pandas as pd
import vectorbt as vbt
import pandas_ta as ta
import numpy as np

In [4]:
def wavetrend(high, low, close, n1, n2, os, ob):
    ap = (high + low + close) / 3.0  # HLC3
    ap = pd.DataFrame(ap)
    esa = ap.ewm(span=n1, adjust=False).mean()  # EMA
    d = (abs(ap - esa)).ewm(span=n1, adjust=False).mean()
    ci = (ap - esa) / (0.015 * d)
    wt1 = ci.ewm(span=n2, adjust=False).mean()
    wt2 = wt1.rolling(4).mean()
    trend = np.where(wt1 < os, -1, 0)
    trend = np.where(wt1 > ob, 1, trend)
    return wt1, wt2, trend

Wavetrend = vbt.IndicatorFactory(
    class_name='Wavetrend',
    short_name='wt',
    input_names=['high', 'low', 'close'],
    param_names=['n1', 'n2', 'os', 'ob'],
    output_names=['wt1', 'wt2', 'trend'], ).from_apply_func(wavetrend)

In [6]:
symbols = ['BTCUSDT','ETHUSDT', 'XRPUSDT', 'LTCUSDT', 'BNBUSDT','SOLUSDT']

interval = '1h'
df = vbt.BinanceData.download(symbols,start='12 months ago UTC', interval=interval).concat()

0it [00:00, ?it/s]

0it [00:00, ?it/s]

0it [00:00, ?it/s]

0it [00:00, ?it/s]

0it [00:00, ?it/s]

0it [00:00, ?it/s]

**WAVETREND 1**

In [ ]:
n1 = np.arange(10,30,1)

n2 = 14

wt = Wavetrend.run(df['High'],df['Low'],df['Close'],n1=n1, n2=n2)

In [ ]:
entries = wt.wt1_crossed_above(wt.wt2)
exits = wt.wt1_crossed_below(wt.wt2)

vbt.settings.portfolio['fees'] = 0.00075
vbt.settings.portfolio['sl_stop'] = 0.245
vbt.settings.portfolio['init_cash'] = 1000
vbt.settings.portfolio['freq'] = interval
vbt.settings.set_theme("dark")

pf1 = vbt.Portfolio.from_signals(df['Close'], entries, exits)
cheight, cwidth = 600, 800
#plot2 = pf1.value().vbt.plot(height=cheight, width=cwidth).show()
#plot1 = pf1.plot_cum_returns(height=cheight, width=cwidth,group_by=True).show()
print(pf1.total_return())
print('')
print(pf1.total_benchmark_return(group_by='symbol'))
#print(pf1.stats(group_by=True))
#print(pf1.trades.records_readable)

In [ ]:
pf1.trades.wrapper.columns

In [ ]:
pf1.stats(column=(10,14,'LTCUSDT'))

In [ ]:
pf1.trades.records_readable

In [ ]:
pf1.plot(height=900, width=1000,column=(24,21,'LTCUSDT')).show()

In [ ]:
pf1.trades.plot(height=700, width=1000,column=(24,21,'LTCUSDT')).show()

In [ ]:
tr = pd.DataFrame(pf1.total_return())
tr['total_return'].groupby('symbol').nlargest()

WAVETREND 2


In [7]:
n1 = np.arange(10,40,1)
n2 = np.arange(10,40,1)
ob = 60
os = -75
wt =  Wavetrend.run(df['High'],df['Low'],df['Close'],n1=n1, n2=n2, os=os, ob=ob, param_product = True)
cmo = vbt.pandas_ta('cmo').run(df['Close'], lenght=14, talin=False)

In [6]:
obLevel1 = 60
obLevel2 = 53
osLevel1 = -75
osLevel2 = -53

In [8]:
entries2 = wt.trend_below(0)
exits2 = wt.trend_above(0)

vbt.settings.set_theme("dark")

pf2 = vbt.Portfolio.from_signals(
    close=df['Close'],
    entries = entries2,
    exits=exits2,
    sl_stop=0.06,
    fees=0.00075,
    freq=interval,
    init_cash=1000,  
    )

cheight, cwidth = 600, 800
#plot2 = pf2.value().vbt.plot(height=cheight, width=cwidth).show()
#plot1 = pf2.plot_cum_returns(height=cheight, width=cwidth,group_by=True).show()
#print(pf2.total_return())
print('')
#print(pf2.total_benchmark_return(group_by='symbol'))
#print(pf2.stats(group_by=True))
#print(pf2.trades.records_readable)

In [9]:
tr2 = pd.DataFrame(pf2.total_return())
tr2['total_return'].groupby('symbol').nlargest(3)

symbol   wt_n1  wt_n2  wt_os  wt_ob  symbol 
BNBUSDT  15     34     -75    60     BNBUSDT    1.470698
         13     30     -75    60     BNBUSDT    1.447112
         14     32     -75    60     BNBUSDT    1.325836
BTCUSDT  13     16     -75    60     BTCUSDT    1.492254
         23     17     -75    60     BTCUSDT    1.401692
         17     18     -75    60     BTCUSDT    1.311591
ETHUSDT  34     15     -75    60     ETHUSDT    0.854391
         35     15     -75    60     ETHUSDT    0.843312
         20     27     -75    60     ETHUSDT    0.812616
LTCUSDT  15     19     -75    60     LTCUSDT    1.611111
                18     -75    60     LTCUSDT    1.607087
         16     19     -75    60     LTCUSDT    1.466174
SOLUSDT  14     19     -75    60     SOLUSDT    3.681037
         15     20     -75    60     SOLUSDT    3.630434
                22     -75    60     SOLUSDT    3.506925
XRPUSDT  12     20     -75    60     XRPUSDT    2.286985
         20     32     -75    60     XRPUSD

In [10]:
pf2.total_benchmark_return().groupby('symbol').max()

symbol
BNBUSDT   -0.275493
BTCUSDT   -0.244278
ETHUSDT    0.287150
LTCUSDT   -0.585112
SOLUSDT    2.185025
XRPUSDT   -0.459978
Name: total_benchmark_return, dtype: float64

In [ ]:
pf2.trades.wrapper.columns

In [11]:
pf2.stats(column=(12,20,-75,60,'XRPUSDT'))

Start                         2021-04-21 15:00:00+00:00
End                           2022-04-21 14:00:00+00:00
Period                                364 days 15:00:00
Start Value                                      1000.0
End Value                                   3286.984614
Total Return [%]                             228.698461
Benchmark Return [%]                          -45.99783
Max Gross Exposure [%]                            100.0
Total Fees Paid                              107.588169
Max Drawdown [%]                              25.966441
Max Drawdown Duration                  64 days 15:00:00
Total Trades                                         33
Total Closed Trades                                  32
Total Open Trades                                     1
Open Trade PnL                                 94.47507
Win Rate [%]                                      56.25
Best Trade [%]                                46.448221
Worst Trade [%]                               -9

In [12]:
pf2.plot(height=900, width=1000,column=(12,20,-75,60,'XRPUSDT')).show()

In [13]:
pf2.plot_trades(height=800, width=1000,column=(12,20,-75,60,'XRPUSDT')).show()

**WAVETREND 3**

In [ ]:
n1 = np.arange(10,40,1)
n2 = np.arange(10,40,1)
wt =  Wavetrend.run(df['High'],df['Low'],df['Close'],n1=n1, n2=n2, param_product = True)
cmo = vbt.pandas_ta('cmo').run(df['Close'], lenght=14, talin=False)

In [ ]:
obLevel1 = 60
obLevel2 = 53
osLevel1 = -60
osLevel2 = -53

In [ ]:
entries3 = wt.wt1_crossed_below(osLevel1)
exits3 = wt.wt1_crossed_above(obLevel1)

vbt.settings.set_theme("dark")

pf3 = vbt.Portfolio.from_signals(
    close=df['Close'],
    entries = entries2,
    tp_stop=0.1,
    sl_stop=0.08,
    fees=0.00075,
    freq=interval,
    init_cash=1000,  
    )

tr3 = pd.DataFrame(pf3.total_return())
tr3['total_return'].groupby('symbol').nlargest(2)

symbol    wavetrend_n1  wavetrend_n2  symbol  
ADAUSDT   17            13            ADAUSDT     0.660454
          18            12            ADAUSDT     0.596573
BNBUSDT   15            10            BNBUSDT     1.089446
          10            11            BNBUSDT     1.026451
BTCUSDT   18            27            BTCUSDT     0.380410
          11            24            BTCUSDT     0.359306
ETHUSDT   17            16            ETHUSDT     1.924050
          16            16            ETHUSDT     1.570168
LTCUSDT   39            19            LTCUSDT     1.057047
          10            37            LTCUSDT     0.963102
LUNAUSDT  19            15            LUNAUSDT    1.628946
          29            10            LUNAUSDT    1.623444
XRPUSDT   38            23            XRPUSDT     0.675595
          12            38            XRPUSDT     0.574483
Name: total_return, dtype: float64

In [ ]:
pf3.plot(height=900, width=1000,column=(17,16,'ETHUSDT')).show()

**FINDING THE BESt PARAMETERS**

In [ ]:
best_parameters = []
n11 = range(10, 40, 1)
n22 = range(10, 40, 1)
sl_losses = [0.04, 0.06, 0.08, 0.1]

for n1 in n11:
  for n2 in n22:
    for sl in sl_losses:
      wt = Wavetrend.run(df['High'],df['Low'],df['Close'],n1=n1, n2=n2)
      entries3 = wt.wt2_crossed_below(osLevel2)
      exits3 = wt.wt2_crossed_above(obLevel2)
      pf = vbt.Portfolio.from_signals(
                        sl_stop=sl,
                        fees=0.00075,
                        freq=interval,
                        init_cash=1000,        
                        close=df['Close'],
                        entries=entries3,
                        exits=exits3,
                    )
      stats = pf.stats(column=(n1,n2,'LTCUSDT'))
      if stats['Profit Factor'] > 1.3:
        print(f'n1: {n1}, n2: {n2}, sl: {sl}, return: {stats["Total Return [%]"]}')
        best_parameters.append({
                            'n1': n1,
                            'n2': n2,
                            'sl': sl,
                            'stat': stats
                        })

In [ ]:
best_parameters = []
n11 = [10,21]
n22 = [10,36]
sl_losses = [0.04, 0.06, 0.08, 0.1]
trail = [0.01, 0.03, 0.05, 0.07]
tp_profits = [0.02, 0.04, 0.06, 0.08, 0.1, 0.15]
for n1 in n11:
  for n2 in n22:
    for sl in sl_losses:
      for tr in trail:
        for tp in tp_profits:
          wt = Wavetrend.run(df['High'],df['Low'],df['Close'],n1=n1, n2=n2)
          entries3 = wt.wt1_crossed_below(osLevel1)
          pf = vbt.Portfolio.from_signals(
                        sl_stop=sl,
                        tp_stop=tp,
                        sl_trail=tr,
                        fees=0.00075,
                        freq=interval,
                        init_cash=1000,        
                        close=df['Close'],
                        entries=entries3,
                    )
          stats = pf.stats(column=(n1,n2,'LTCUSDT'))
          if stats['Profit Factor'] > 1.35:
            print(f'n1: {n1}, n2: {n2}, sl: {sl}, tp: {tp}, tr: {tr}, return: {stats["Total Return [%]"]}')
            best_parameters.append({
                            'n1': n1,
                            'n2': n2,
                            'sl': sl,
                            'tr': tr,
                            'tp': tp,
                            'stat': stats
                        })